# Actual CISI-IMC 
For Fig2ab, Extended Data Fig.7

This code decompress the actual composite data from evaluaiton dataset.

Additionally, reweighted A (Ahat) was computed and also used for decompression.

Individual protein expressions for ground truth and decompressed (with Ahat) were saved as tiff images. Cell mask images have to be provided for this. 

## Import libraries

In [1]:
# Import libraries
import anndata as ad
from pathlib import Path
import os
import numpy as np
import pandas as pd
import errno

## setup

In [2]:
# Import system libraries to configure code directory as module
from os.path import dirname, abspath, join
import sys

# Find code directory relative to our directory
THIS_DIR = dirname('__file__')
CODE_DIR = abspath(join(THIS_DIR, '..', 'code'))
# Add code directory to systems paths
sys.path.append(CODE_DIR)

In [3]:
# Import fnc.
from smaf import smaf
from utils import correlations, analyse_decoding, sparse_decode_blocks_lasso,analyse_U_W, is_valid_file, iterative_decode_W_A_lasso

## Input data and specify output
sce (containing Xtrue and Y), A, and U are loaded. 

Note 1: sce (Anndata: cells x channels) has sce.var.channel_type to distinguish ground truth and CISI channels.

sce.var.index for ground truth is named as "GT_(protein name)"

sce.var.index for CISI channels is named as "CISI_(channel no)"

Note 2: A (channel x protein) should be csv file and its index (CISI_channels) has to match channel ids in sce.var.index



In [4]:
# Path to sce data
data_path = Path('/mnt/projects/data')
sce_path = Path(os.path.join(data_path,'0_preprocess_th186/processed/filtered_sce.h5ad'))

# Path to U and phi
A_path = Path(os.path.join(data_path,'0_preprocess_th186/phi.csv'))
U_path = Path(os.path.join(data_path,'10_Finalize_U/U/U_final.csv'))


# Output Dir
EXP_name = 'publication/9_CISI_actual'
out_path = Path(os.path.join(data_path, EXP_name))
out_path_Uev = Path(os.path.join(data_path, EXP_name, "U_from_evaluaiton_dataset"))
out_path_sce = Path(os.path.join(data_path, EXP_name, "sce"))
# Create output directory if it doesn't exist
out_path.mkdir(parents=True, exist_ok=True)
out_path_Uev.mkdir(parents=True, exist_ok=True)
out_path_sce.mkdir(parents=True, exist_ok=True)

In [5]:
# Check that input files/dictionary exist
if not is_valid_file(sce_path, ['.h5ad']):
    # If file is not found, throw error
    raise FileNotFoundError(errno.ENOENT, os.strerror(errno.ENOENT),sce_path)

In [6]:
# read sce 
sce = ad.read_h5ad(sce_path)

# remove other tissues than new 5 tumors
sce = sce[sce.obs.tissue.isin(["Mealnoma","ccRCC","LungAd","LungSq","OvaryC"]),:] 

# subset ground truth channel and CISI channel
sce_gt = sce[:,sce.var.channel_type == "GT"]
sce_cisi = sce[:,sce.var.channel_type == "CISI"]
sce_gt,sce_cisi

(View of AnnData object with n_obs × n_vars = 98713 × 16
     obs: 'sample_id', 'ObjectNumber', 'width_px', 'height_px', 'ROI_fullname', 'slide', 'tissue', 'ROI_per_tissue'
     var: 'channel', 'name', 'keep', 'ilastik', 'deepcell', 'cellpose', 'channel_type'
     uns: 'X_name',
 View of AnnData object with n_obs × n_vars = 98713 × 8
     obs: 'sample_id', 'ObjectNumber', 'width_px', 'height_px', 'ROI_fullname', 'slide', 'tissue', 'ROI_per_tissue'
     var: 'channel', 'name', 'keep', 'ilastik', 'deepcell', 'cellpose', 'channel_type'
     uns: 'X_name')

### Set the standardized proteins names across Xtrue, A and U
We reference the order from ground truth X

In [7]:
# set the order of genes from sce_gt (ground truth X)
gene_order = [gene.split("_")[1].replace("GranB","GranzymeB") for gene in sce_gt.var.index]
gene_order

['MPO',
 'SMA',
 'CD38',
 'CD303',
 'CD68',
 'CD163',
 'CD20',
 'CD3',
 'DC-LAMP',
 'CD7',
 'GranzymeB',
 'Ki-67',
 'CD8a',
 'FOXP3',
 'CD4',
 'CD31']

In [8]:
# rename the protein names
sce_gt.var.index = gene_order

### Read U and reindex according to the order of proteins

In [9]:
# read U and reorder genes
U = pd.read_csv(U_path, index_col=0)
U = U.reindex(gene_order)
U

,1,2,3,4,5,6,7,8,9,10,...,12,13,14,15,16,17,18,19,20,21
MPO,0.000000,0.000000,0.011682,0.0,0.999433,0.000000,0.0,0.0,0.000000,0.000000,...,0.0,0.076374,0.000000,0.000000,0.000000,0.0,0.000000,0.033007,0.255365,0.0
SMA,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,...,0.0,0.951139,0.999966,0.000000,0.000000,0.0,0.000000,0.000000,0.081692,0.0
CD38,0.000000,0.998952,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.990714,0.000000,...,0.0,0.120189,0.008211,0.289768,0.088353,0.0,0.000000,0.115110,0.418095,0.0
CD303,0.000000,0.000000,0.000000,0.0,0.033664,0.000000,1.0,0.0,0.135960,0.000000,...,0.0,0.199299,0.000000,0.000000,0.000000,0.0,0.000000,0.055825,0.560318,0.0
CD68,0.000000,0.000000,0.999932,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,...,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.102589,0.0
CD163,0.000000,0.000000,0.000000,0.0,0.000000,0.995775,0.0,0.0,0.000000,0.000000,...,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.016078,0.0
CD20,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,...,0.0,0.000000,0.000000,0.000000,0.995961,0.0,0.000000,0.000000,0.000000,0.0
CD3,0.292363,0.000000,0.000000,0.0,0.000000,0.000000,0.0,1.0,0.000000,0.081871,...,0.0,0.000000,0.000000,0.000000,0.015955,0.0,0.815970,0.000000,0.084003,0.0
DC-LAMP,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,...,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0
CD7,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.996643,...,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.052157,0.0


### Read A and reindex according to the order of CISI_channels

In [10]:
# read A and reorder genes. 
A = pd.read_csv(A_path, index_col=0)
A = A[gene_order]
# check order of CISI channels. 
cisi_ch_order = sce_cisi.var.index.tolist()
A = A.reindex(cisi_ch_order)
A

,MPO,SMA,CD38,CD303,CD68,CD163,CD20,CD3,DC-LAMP,CD7,GranzymeB,Ki-67,CD8a,FOXP3,CD4,CD31
CISI_1,0,0.0,0.0,0.0,0.0,0.0,0.5,0.0,0.0,1,0,0.0,0.5,0.0,0.0,0.0
CISI_2,0,0.0,0.0,0.0,0.5,0.0,0.5,0.0,0.5,0,0,0.0,0.0,0.0,0.5,0.0
CISI_3,0,0.0,0.5,0.0,0.5,0.0,0.0,0.5,0.0,0,0,0.5,0.0,0.0,0.0,0.0
CISI_4,0,0.0,0.0,0.0,0.0,0.5,0.0,0.0,0.5,0,0,0.5,0.0,0.5,0.0,0.0
CISI_5,0,0.5,0.0,0.0,0.0,0.5,0.0,0.0,0.0,0,0,0.0,0.5,0.0,0.5,0.5
CISI_6,1,0.0,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0.0,0.0,0.0,0.0,0.5
CISI_7,0,0.5,0.0,0.5,0.0,0.0,0.0,0.5,0.0,0,1,0.0,0.0,0.0,0.0,0.0
CISI_8,0,0.0,0.0,0.5,0.0,0.0,0.0,0.0,0.0,0,0,0.0,0.0,0.5,0.0,0.0


## Calculate U from evaluation dataset for a comparison

In [11]:
# calculate U from evaluation dataset

## Set params for U
# set variables
d = 80
maxItr = 100
nthread = -1
methodW = 'lasso'
ldaU = 0.02
nblocksW_lasso = 1
ldaW = 0.02


# calc U
U_ev,W,X = smaf(sce_gt,d,maxItr,methodW,ldaU, ldaW=ldaW, THREADS=nthread,  X_normalization='paper_norm',
          num_blocks_W=nblocksW_lasso, num_blocks_U=1, layer=None, Normalize_U=True, saveItr=False) 
# get SMAF results
res_U, coln_U = analyse_U_W(U_ev, W, X)

# save U_ev 
pd.DataFrame(U_ev, columns=list(range(1, U_ev.shape[1]+1)),index=gene_order).to_csv(
    os.path.join(out_path_Uev, 'U.csv'))

# Transform U results into DF
df = pd.DataFrame(res_U).T ## .T added since it's 1D DF
df.columns = coln_U
# save U results as csv
df.to_csv(path_or_buf=os.path.join(out_path_Uev, 'result_U.csv'))

Initialized U, W with NMF, SMAF maxItr =  100


## Parameters for reweighting A

In [12]:
# recalculate A_weights
ldaW_dc = 0.02  # error tolerance when calculating W
ldaA = 0.0002   # error tolerance when calculating A
nblocks = 200   # no of blocks of cells when calculating W
maxItr = 6      # no of iteration

## Decompress with 4 conditions and save results
1. Actual: Using Y from actual CISI-IMC data, A without reweighting, U from training dataset.

2. U_from_evaluationdataset: Using Y from actual CISI-IMC data, A without reweighting, U from evaluation dataset.

3. Y_simulated: Using Y simulated from ground truth CISI-IMC data, A without reweighting, U from training dataset.

4. Actual_Ahat: Using Y from actual CISI-IMC data, reweighted A, U from training dataset.
### Prepare ground truth X and save

In [13]:
# prepare ground truth X
Xtrue = (sce_gt.X).T
# save X data
## remove all layers other than counts
for k in list(sce_gt.layers.keys()):
    del sce_gt.layers[k]
    
# save ground truth X as h5ad
sce_gt.write(os.path.join(out_path_sce, 'sce_gt.h5ad'))


### Decompress and save decoded X (Xhat) as h5ad 


In [14]:
# Results container
df_list = []
df_genewise_list = []

# set composite data
Y = (sce_cisi.X).T
Ysim = np.array(A).dot(Xtrue) # simulated_Y

# conditions
conds = ["Actual", "U_from_evaluationdataset", "Y_simulated","Actual_Ahat"]
Ys = [Y, Y, Ysim, Y]
Us = [U, U_ev, U, U]

for cond, cur_Y, cur_U in zip(conds,Ys,Us):
    if cond == "Actual_Ahat":
        # Decode
        W, Ahat = iterative_decode_W_A_lasso(cur_Y, np.array(A), np.array(cur_U), ldaW_dc, ldaA, maxItr, -1, nblocks, eachItr = False)
        Xhat = np.array(cur_U).dot(W)
        Xhat[np.isnan(Xhat)] = 0

        # analyze results
        res, coln, details = analyse_decoding(Ahat,cur_U,W,Xtrue, Xhat, name="", detail=True)
    else:
        # Decode
        W = sparse_decode_blocks_lasso(Y = cur_Y, D = np.array(A).dot(np.array(cur_U)), lda = ldaW_dc, numThreads = -1, num_blocks = nblocks)
        Xhat = np.array(cur_U).dot(W)
        Xhat[np.isnan(Xhat)] = 0

        # analyze results
        res, coln, details = analyse_decoding(A,cur_U,W,Xtrue, Xhat, name="", detail=True)

    # general results
    df = pd.DataFrame(res).T # .T added since it's 1D DF
    df.columns = coln
    df_list.append(df)

    # genewise results
    df_genewise = pd.DataFrame(details[0]).T
    df_genewise.columns = gene_order
    df_genewise_list.append(df_genewise)

    # save decoded X as h5ad (copy col/row Data from X_gt)
    sce_dc = sce_gt.copy()
    sce_dc.X = Xhat.T
    sce_dc.write(os.path.join(out_path_sce, 'sce_dc_{}.h5ad'.format(cond)))


/home/ubuntu/miniconda3/envs/test_cisi_imc_env/lib/python3.10/site-packages/scipy/spatial/distance.py:622: RuntimeWarning: invalid value encountered in double_scalars
  dist = 1.0 - uv / np.sqrt(uu * vv)
/home/ubuntu/miniconda3/envs/test_cisi_imc_env/lib/python3.10/site-packages/scipy/spatial/distance.py:622: RuntimeWarning: invalid value encountered in double_scalars
  dist = 1.0 - uv / np.sqrt(uu * vv)
/home/ubuntu/miniconda3/envs/test_cisi_imc_env/lib/python3.10/site-packages/scipy/spatial/distance.py:622: RuntimeWarning: invalid value encountered in double_scalars
  dist = 1.0 - uv / np.sqrt(uu * vv)
/home/ubuntu/miniconda3/envs/test_cisi_imc_env/lib/python3.10/site-packages/scipy/spatial/distance.py:622: RuntimeWarning: invalid value encountered in double_scalars
  dist = 1.0 - uv / np.sqrt(uu * vv)


## save analysis results

In [15]:
# combine DF
df = pd.concat(df_list)
df.index = conds
df.to_csv(path_or_buf=os.path.join(out_path, 'result_summary.csv'))

df_genewise = pd.concat(df_genewise_list)
df_genewise.index = conds
df_genewise.to_csv(path_or_buf=os.path.join(out_path, 'result_genewise.csv'))

## save A, Ahat (reweighted A), and U

In [16]:
# save A Ahat U
Ahat_save = A*1
Ahat_save.iloc[:,:] = Ahat
Ahat_save.to_csv(path_or_buf=os.path.join(out_path, 'Ahat.csv'))
A.to_csv(path_or_buf=os.path.join(out_path, 'A.csv'))
U.to_csv(path_or_buf=os.path.join(out_path, 'U.csv'))

## Correlations of composite data (actual vs simulated)

In [17]:
# get simulated Y using Ahat
Ysim_ahat = Ahat.dot(Xtrue)

# calculate channelwise correlations
_, corr1 = correlations(Y, Ysim)      # corr1 = [(cellwise_correlation), (channelwise_correlation))] 
_, corr2 = correlations(Y, Ysim_ahat)

# save as csv
df_cchcorr = pd.DataFrame({"actual_vs_simulated_A":corr1[0], "actual_vs_simulated_Ahat":corr2[0]})
df_cchcorr.to_csv(path_or_buf=os.path.join(out_path, 'result_cisich_corr.csv'))


/home/ubuntu/miniconda3/envs/test_cisi_imc_env/lib/python3.10/site-packages/scipy/spatial/distance.py:622: RuntimeWarning: invalid value encountered in double_scalars
  dist = 1.0 - uv / np.sqrt(uu * vv)




## save Y

In [18]:
# save Y as h5ad
## remove all layers other than counts
for k in list(sce_cisi.layers.keys()):
    del sce_cisi.layers[k]
    
# save Y as h5ad
sce_cisi.write(os.path.join(out_path_sce, 'sce_cisi.h5ad'))

# save Ysim as h5ad (copy col/row Data from Y)
sce_cisi_sim = sce_cisi.copy()
sce_cisi_sim.X = Ysim.T
sce_cisi_sim.write(os.path.join(out_path_sce,'sce_cisi_sim.h5ad'))

# save Ysim as h5ad (copy col/row Data from Y)
sce_cisi_sim_ahat = sce_cisi.copy()
sce_cisi_sim_ahat.X = Ysim_ahat.T
sce_cisi_sim_ahat.write(os.path.join(out_path_sce,'sce_cisi_sim_ahat.h5ad'))

# Save cell images as tiff (optional)
Given the cell masks (filename should match the anndata.obs.sample_id), produces cell images for each ROI as stacked tifffile. Each tiff contains images for all markers.
## load libraries


In [20]:
import glob
import skimage
import tifffile

/home/ubuntu/miniconda3/envs/test_cisi_imc_env/lib/python3.10/site-packages/paramiko/transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,


## Input data and set output

In [21]:
# path to data
data_path = Path('/mnt/projects/data')

# path to cellmask data
mask_paths = glob.glob(os.path.join(data_path,'0_preprocess_th186/processed/masks/*.tiff'))

# path to h5ad data
gt_path = os.path.join(data_path,'publicaiton/9_CISI_actual/sce/sce_gt.h5ad')
dc_path = os.path.join(data_path,'publicaiton/9_CISI_actual/sce/sce_dc_Actual_Ahat.h5ad')

# set output dir
out_path_tiffs = Path(os.path.join(out_path, 'tiffs'))
# Create output directory if it doesn't exist
out_path_tiffs.mkdir(parents=True, exist_ok=True)

In [22]:
# read sce
sce_gt = ad.read_h5ad(gt_path)
sce_dc = ad.read_h5ad(dc_path)

['/mnt/projects/data/0_preprocess_th186/processed/masks/20230726_TsH_th186_BreastC1_003.tiff',
 '/mnt/projects/data/0_preprocess_th186/processed/masks/20230726_TsH_th186_BreastC1_002.tiff',
 '/mnt/projects/data/0_preprocess_th186/processed/masks/20230726_TsH_th186_BreastC1_001.tiff',
 '/mnt/projects/data/0_preprocess_th186/processed/masks/20230726_TsH_th186_BreastC2_001.tiff',
 '/mnt/projects/data/0_preprocess_th186/processed/masks/20230726_TsH_th186_BreastC2_002.tiff',
 '/mnt/projects/data/0_preprocess_th186/processed/masks/20230726_TsH_th186_BreastC2_003.tiff',
 '/mnt/projects/data/0_preprocess_th186/processed/masks/20230724_TsH_th186_ccRCC_003.tiff',
 '/mnt/projects/data/0_preprocess_th186/processed/masks/20230724_TsH_th186_ccRCC_002.tiff',
 '/mnt/projects/data/0_preprocess_th186/processed/masks/20230724_TsH_th186_ccRCC_001.tiff',
 '/mnt/projects/data/0_preprocess_th186/processed/masks/20230725_TsH_th186_LungAd_001.tiff',
 '/mnt/projects/data/0_preprocess_th186/processed/masks/20230

## Get cell images and save as tiff
Put the X (or Xhat) values onto cell mask image to get cell images. Value for each cell in cell mask image matches the cell_id in X (or Xhat).

In [36]:
# get list for iteration
tissue_list = sce_gt.obs.tissue.unique().tolist()

In [37]:
## automatically creating tiff images for each tissue, each ROI

for cur_tissue in tissue_list:
    for cur_ROI in sce_gt[sce_gt.obs.tissue == cur_tissue].obs.ROI_per_tissue.unique():      
        # set current sample_id
        cur_sample_id = "".join(sce_gt[(sce_gt.obs.tissue == cur_tissue) & (sce_gt.obs.ROI_per_tissue == cur_ROI)].obs.sample_id.unique())
        # obtain np array for current ROI/sample_id
        gt = sce_gt[sce_gt.obs.sample_id == cur_sample_id].X
        dc = sce_dc[sce_dc.obs.sample_id == cur_sample_id].X
        # get cell mask matching current sample_id/ROI
        cur_mask = skimage.io.imread([mask_path for mask_path in mask_paths if cur_sample_id in mask_path][0])
        # add 0 at idx=0 for non-cell pixels
        gt = np.insert(gt, 0, 0, axis=0)
        dc = np.insert(dc, 0, 0, axis=0)
        # get gt and dc images
        gt_images = np.moveaxis(gt[cur_mask], -1, 0).astype('float32')
        dc_images = np.moveaxis(dc[cur_mask], -1, 0).astype('float32')
        # save tiffs
        tifffile.imsave(os.path.join(out_path_tiffs,'{}_{}_gt.tiff'.format(cur_tissue,cur_ROI)), gt_images)
        tifffile.imsave(os.path.join(out_path_tiffs,'{}_{}_dc.tiff'.format(cur_tissue,cur_ROI)), dc_images)